In [1]:
import numpy as np
import pandas as pd
import glob
import re
from sklearn.utils import shuffle
import torch
import torch.nn.functional as F
import torch.nn as nn
from sklearn.metrics import confusion_matrix, f1_score
import torch.optim as optim
from sklearn.model_selection import train_test_split

In [2]:
HamDirs = pd.Series(np.array(glob.glob('datasets/email/ham/*/*/*')))
SpamDirs = pd.Series(np.array(glob.glob('datasets/email/spam/*/*/*')))

SpamDirs.shape, HamDirs.shape

((21103,), (19088,))

In [3]:
SpamDirs.head()

0    datasets/email/spam/SH/HP/prodmsg.2.431622.200...
1    datasets/email/spam/SH/HP/prodmsg.2.431958.200...
2    datasets/email/spam/SH/HP/prodmsg.2.430984.200...
3    datasets/email/spam/SH/HP/prodmsg.2.444740.200...
4    datasets/email/spam/SH/HP/prodmsg.2.447580.200...
dtype: object

Email content

In [4]:
with open(SpamDirs[1], encoding='windows-1252') as f:
    file_content = f.read()

print(file_content)

Received: from c2b03-016.milare-tv.ne.jp (HELO @smtp) (219.124.83.16)
  by projecthoneypotmailserver with SMTP; 25 Jun 2005 10:59:32 -0000
Received: from 209.239.43.9 (helo=MANN)
	by smtp with esmtpa (Exim 4.50b (OpenBSD))
	id J85Gz020412586
	for projecthoneypot@projecthoneypot.org; Sat, 25 Jun 2005 10:55:18 +0000
Date: Sat, 25 Jun 2005 10:55:18 +0000
From: "Kristina" <nc46f7486j9@direcway.com>
X-Mailer: Java-x Mailer v1.3b
Reply-To: "Kristina" <nc46f7486j9@direcway.com>
X-Priority: 3 (Normal)
Message-ID: <t4hcwe.20050606213053@localhost>
To: projecthoneypot@projecthoneypot.org
Subject: Any med for your girl to be happy! 
MIME-Version: 1.0
Content-Type: text/html; charset=Windows-1252
Content-Transfer-Encoding: 8bit

<html>
<body>
Your girl is unsatisfied with your potency? Don't wait until she finds another men!<br><br>

Click <a href="http://rbccrl.skysalononline.info/?mpkuwixwtqtyudbwwczpohvwdwg">here</a> to choose from a great variety of LlCENSED love t@bs! Best pri$es ,fast shippi

#### Get Content (With Data Cleaning)

For Header Handling

In [5]:
# For Header Filter

MailHeaders = ["received", 'mime-version', 'reply-to',
             "date", "content-transfer-encoding", "message-id", "x-from", "x-to", "x-cc", "x-bcc",
             "x-origin", "x-filename", "x-priority", "x-msmail-priority", "organization", "x-mailer"]

EmailHeaderRegex = "from:\s(?P<from>.*)?[^^]*subject:\s(?P<subject>.*)?[^^]*content-type:\s(?P<content_type>.*)?"

In [6]:
# Extract Mail

def getContent(filePath):
    
    with open(filePath, encoding='windows-1252') as f:
        contents = f.read()
        
        # Data Cleaning
        contents = contents.lower()
        contents = re.sub("<[^<>]+>", " __tag__ ", contents)
        contents = re.sub("[0-9]+", "number", contents)
        contents = re.sub("(http|https)://[^\s]*", "httpaddr", contents)
        contents = re.sub("[^\s]+@[^\s]+", "emailaddr", contents)
        contents = re.sub("[$]+", "dollar", contents)
        contents = re.sub("(" + ":|".join(MailHeaders)  + ")+", "header:", contents)
        
    Email_Headers = re.findall(EmailHeaderRegex, contents)
    (Email_From, Subject, Content_Type) = Email_Headers[0] if len(Email_Headers) else ("", "", "") 
    
    __SEPERATOR__ = " __SEPERATOR__ "
    
    BodyRegex = "(?:[^^]*)(?:header\s*:.*\n)([^^]*)"  
    Body = re.findall(BodyRegex, contents)
    Body = Body[0] if len(Body) else ""
    
    
    MailMessage = re.split("-----original message-----", contents)[0]
    content = Email_From + __SEPERATOR__ + Subject + __SEPERATOR__ + Content_Type + __SEPERATOR__ + "\n" + Body
    
    contentSeries = pd.Series([filePath, content, Body, Email_From, Subject, Content_Type], 
                              index=["filePath", "content", "body", "from", "subject", "content_type"]);
    
    return contentSeries

### We will use initial 700 records for training

In [7]:
HamDf = HamDirs[:700].apply(getContent)
SpamDf = SpamDirs[:700].apply(getContent)
HamDf["label"] = 0
SpamDf["label"] = 1

SpamDf.head()

,filePath,content,body,from,subject,content_type,label
0,datasets/email/spam/SH/HP/prodmsg.2.431622.200...,"""goto my meeting"" __tag__ __SEPERATOR__ the ...",\n __tag__ \n\n __tag__ \n __tag__ \n __tag__ ...,"""goto my meeting"" __tag__",the next generation in online meetings!,text/html,1
1,datasets/email/spam/SH/HP/prodmsg.2.431958.200...,"""kristina"" __tag__ __SEPERATOR__ any med for...",\n __tag__ \n __tag__ \nyour girl is unsatisfi...,"""kristina"" __tag__",any med for your girl to be happy!,text/html; charset=windows-number,1
2,datasets/email/spam/SH/HP/prodmsg.2.430984.200...,__SEPERATOR__ __SEPERATOR__ __SEPERATOR__ \...,\n __tag__ \n __tag__ \n __tag__ numberth rebu...,,,,1
3,datasets/email/spam/SH/HP/prodmsg.2.444740.200...,"""ardith"" __tag__ __SEPERATOR__ save your mon...",\n __tag__ \n __tag__ \nneed in software for y...,"""ardith"" __tag__",save your money by getting an oem software!,text/html; charset=windows-number,1
4,datasets/email/spam/SH/HP/prodmsg.2.447580.200...,mail delivery subsystem __tag__ __SEPERATOR_...,\n\n,mail delivery subsystem __tag__,mail message detected as spam,"text/plain; charset=""utf-number""",1


In [8]:
print(SpamDf.loc[1, "body"])


 __tag__ 
 __tag__ 
your girl is unsatisfied with your potency? don't wait until she finds another men! __tag__  __tag__ 

click  __tag__ here __tag__  to choose from a great variety of llcensed love emailaddr best pridollares ,fast shipping and guaranteed effect! here you buy it riqht from warehouse! __tag__  __tag__ 

the store is verifled by bbb and approved by visa! __tag__ 

 __tag__ 
 __tag__ 



In [9]:
print(SpamDf.loc[1, "content"])

"kristina"  __tag__  __SEPERATOR__ any med for your girl to be happy!  __SEPERATOR__ text/html; charset=windows-number __SEPERATOR__ 

 __tag__ 
 __tag__ 
your girl is unsatisfied with your potency? don't wait until she finds another men! __tag__  __tag__ 

click  __tag__ here __tag__  to choose from a great variety of llcensed love emailaddr best pridollares ,fast shipping and guaranteed effect! here you buy it riqht from warehouse! __tag__  __tag__ 

the store is verifled by bbb and approved by visa! __tag__ 

 __tag__ 
 __tag__ 



##### Mail dataset with label

In [10]:
datasetDf = pd.concat([HamDf, SpamDf], axis = 0, ignore_index = True)
datasetDf = shuffle(datasetDf, random_state= 0)
datasetDf.to_csv("datasets/email/email.csv", index = False)

datasetDf.head()

,filePath,content,body,from,subject,content_type,label
1110,datasets/email/spam/SH/HP/prodmsg.2.431713.200...,"""goto my meeting"" __tag__ __SEPERATOR__ the ...",\n __tag__ \n\n __tag__ \n __tag__ \n __tag__ ...,"""goto my meeting"" __tag__",the next generation in online meetings!,text/html,1
310,datasets/email/ham/lokay-m/personal/142,emailaddr __SEPERATOR__ good job! __SEPERATOR_...,\n\nthanks to the teamwork and dedication of t...,emailaddr,good job!,text/plain; charset=us-ascii,0
776,datasets/email/spam/SH/HP/prodmsg.2.447549.200...,mail delivery system __tag__ __SEPERATOR__ j...,importance: normal\n\n __tag__ \n __tag__ \n _...,mail delivery system __tag__,just to her...,text/html; charset=iso-number-number,1
491,datasets/email/ham/lokay-m/tw_commercial_group...,"emailaddr __SEPERATOR__ tw weekly,number-numbe...",\nplease see attached file and call me if you ...,emailaddr,"tw weekly,number-number-number",text/plain; charset=us-ascii,0
1063,datasets/email/spam/SH/HP/prodmsg.2.437125.200572,"""computer shopping network"" __tag__ __SEPERA...",\n __tag__ \n __tag__ \n\n __tag__ \n\n __tag_...,"""computer shopping network"" __tag__",number's of computer products on sale now!,text/html,1


##### Vocabulary

In [11]:
# Genarate Vocab
from torchtext.vocab import vocab
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
train_iter = datasetDf["content"].to_numpy()

def yield_tokens(data_iter):
    for content in data_iter:
        yield tokenizer(content)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])
vocab.set_default_index(vocab["<unk>"])

##### Tokenization of words (which words are present in content)

In [12]:
# Tokenization
def preprocessData(dfS):
    content = dfS["content"]
    
    tokensIdx = vocab(tokenizer(content))
    
    tokens = np.zeros(len(vocab), dtype="int64")
    
    tokens[tokensIdx] = 1
    
    tokens = tokens.tolist()
    data = pd.Series([dfS["filePath"], tokens], index = ["file", "tokens"])
    
    return data

In [13]:
dataset_tokens_df = datasetDf.apply(preprocessData, axis =1)
dataset_tokens_df["label"] = datasetDf["label"]
# dataset_tokens_df["tokens"] = dataset_tokens_df["tokens"].apply(lambda x: "|".join(map(str, x)))
# dataset_tokens_df.to_csv("datasets/email/emailTokens.csv", index = False)

dataset_tokens_df.head()

,file,tokens,label
1110,datasets/email/spam/SH/HP/prodmsg.2.431713.200...,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, ...",1
310,datasets/email/ham/lokay-m/personal/142,"[0, 0, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, ...",0
776,datasets/email/spam/SH/HP/prodmsg.2.447549.200...,"[0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, ...",1
491,datasets/email/ham/lokay-m/tw_commercial_group...,"[0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, ...",0
1063,datasets/email/spam/SH/HP/prodmsg.2.437125.200572,"[0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 1, ...",1


In [14]:
np.array(dataset_tokens_df.loc[1, "tokens"])[0:500]

array([0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1,
       1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 1, 1, 0, 0,
       0, 0, 0, 1, 1, 0, 0, 0, 1, 1, 0, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0,
       0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 1, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0,

##### Train test split

In [15]:
X = np.array(dataset_tokens_df["tokens"].to_list())
Y = dataset_tokens_df["label"].values


X_train, X_test, y_train, y_test = train_test_split( X, Y, test_size=0.2, random_state=0)

##### Torch model with two Linear layers

In [16]:
model = nn.Sequential(
    nn.Linear(X.shape[1], 20), 
    
    nn.ReLU(),
    nn.Linear(20, 1),
    nn.Sigmoid()
)

# Stochastic gradient descent
optimizer = optim.SGD(model.parameters(), lr=0.1)
loss_fn = nn.BCELoss()

In [17]:
# Training
def train(X, Y, epoches):
    X = torch.FloatTensor(X)
    Y = torch.FloatTensor(Y)
    N = len(Y)
    model.train()
    
    sum_loss = 0
    for epoch in range(epoches):
        
        optimizer.zero_grad()
        output = model(X).squeeze()
        loss = loss_fn(output, Y)
        
        loss.backward()
        optimizer.step()

        sum_loss += float(loss)
        
        if epoch % 100 == 0:
            print("Epoch: {:4d}\t Loss: {}".format(epoch, loss))
        
    
    print("Epoch: {:4d}\t Average Loss: {}".format(epoch, sum_loss / N))
    
    


In [18]:
# For Testing
def test(X, Y):
    X = torch.FloatTensor(X)
    Y = torch.FloatTensor(Y)
    N = len(Y)
    model.eval()
    sum_loss = 0
    
    with torch.no_grad():
        output = model(X).squeeze()
        
        loss = loss_fn(output, Y)
        
        sum_loss += float(loss)
    
    print("loss: {}".format(sum_loss / N))

In [19]:
# For predication
def predict(X):
    y_pred = (model(torch.FloatTensor(X)).squeeze().detach().numpy() > 0.5).astype("int64")
    return y_pred

In [20]:
train(X_train, y_train, 800)

Epoch:    0	 Loss: 0.6965649127960205
Epoch:  100	 Loss: 0.04470809921622276
Epoch:  200	 Loss: 0.017132468521595
Epoch:  300	 Loss: 0.009880070574581623
Epoch:  400	 Loss: 0.006704406347125769
Epoch:  500	 Loss: 0.004970681853592396
Epoch:  600	 Loss: 0.003898353548720479
Epoch:  700	 Loss: 0.0031781515572220087
Epoch:  799	 Average Loss: 0.02544594754643705


In [21]:
test(X_test, y_test)

loss: 5.359051948679345e-05


In [22]:
y_pred = predict(X_test)
confusion_matrix(y_test, y_pred), f1_score(y_test, y_pred)

(array([[137,   0],
        [  2, 141]]),
 0.9929577464788732)

In [23]:
y_pred = predict(X_train)
confusion_matrix(y_train, y_pred)

array([[563,   0],
       [  0, 557]])

### We will test on records after 700

In [24]:
HamDf2 = HamDirs[700:900].apply(getContent)
SpamDf2 = SpamDirs[700:800].apply(getContent)
HamDf2["label"] = 0
SpamDf2["label"] = 1

datasetDf2 = pd.concat([HamDf2, SpamDf2], axis = 0, ignore_index = True)
datasetDf2.head()

,filePath,content,body,from,subject,content_type,label
0,datasets/email/ham/lokay-m/tw_commercial_group...,emailaddr __SEPERATOR__ sun devil to panda lin...,"\nhi beth,\n\taccording to the engineers, mini...",emailaddr,sun devil to panda line pressure,text/plain; charset=us-ascii,0
1,datasets/email/ham/lokay-m/tw_commercial_group...,emailaddr __SEPERATOR__ tw bullets number/numb...,"\nfuel hedging - we hedged another number,numb...",emailaddr,tw bullets number/number,text/plain; charset=us-ascii,0
2,datasets/email/ham/lokay-m/tw_commercial_group...,emailaddr __SEPERATOR__ sun devil fuel __SEPER...,\nben and ron--\n\tfollowing up on our meeting...,emailaddr,sun devil fuel,text/plain; charset=us-ascii,0
3,datasets/email/ham/lokay-m/tw_commercial_group...,emailaddr __SEPERATOR__ red rock contract priv...,\nin response to a concern stan brought up yes...,emailaddr,red rock contract privileged and confidential,text/plain; charset=us-ascii,0
4,datasets/email/ham/lokay-m/tw_commercial_group...,emailaddr __SEPERATOR__ tw bullets number/numb...,\nagave energy - agave purchased a seasonal wi...,emailaddr,tw bullets number/number,text/plain; charset=us-ascii,0


In [25]:
# Shuffle our dataset
datasetDf2 = shuffle(datasetDf2, random_state= 0)
datasetDf2.head()

,filePath,content,body,from,subject,content_type,label
208,datasets/email/spam/SH/HP/prodmsg.2.446010.200...,__tag__ __SEPERATOR__ mentoring from top mar...,\nthis week i showed more than number people\n...,__tag__,mentoring from top mark/eter - no fee,text/plain;,1
188,datasets/email/ham/lokay-m/tw_commercial_group...,emailaddr __SEPERATOR__ project sun devil desc...,\n this document was prepared to supplement et...,emailaddr,project sun devil description,text/plain; charset=us-ascii,0
12,datasets/email/ham/lokay-m/tw_commercial_group...,emailaddr __SEPERATOR__ transport __SEPERATOR_...,\nhey are you in the office today? (thursday ...,emailaddr,transport,text/plain; charset=us-ascii,0
221,datasets/email/spam/SH/HP/prodmsg.2.438579.200574,__SEPERATOR__ __SEPERATOR__ __SEPERATOR__ \...,\n __tag__ \n __tag__ __tag__ \n __tag__ \n _...,,,,1
239,datasets/email/spam/SH/HP/prodmsg.2.430150.200...,"""javier doerr"" __tag__ __SEPERATOR__ luv our...","\nthey are definitely the finest from rolexes,...","""javier doerr"" __tag__",luv our rolexes for the same fea-tures and lov...,text/plain;,1


In [26]:
# Tokenization
dataset_tokens_df2 = datasetDf2.apply(preprocessData, axis =1)
dataset_tokens_df2["label"] = datasetDf2["label"]

In [27]:
X2 = np.array(dataset_tokens_df2["tokens"].to_list())
Y2 = dataset_tokens_df2["label"].values

X_train2, X_test2, y_train2, y_test2 = train_test_split( X2, Y2, test_size=0.2, random_state=0)

In [28]:
y_pred = predict(X_test2)
print(f1_score(y_test2, y_pred))

confusion_matrix(y_test2, y_pred)

1.0


array([[32,  0],
       [ 0, 28]])

In [29]:
y_pred = predict(X_train2)
print(f1_score(y_train2, y_pred))
confusion_matrix(y_train2, y_pred)

1.0


array([[168,   0],
       [  0,  72]])

### Which words are responsible for prediction

In [30]:
a = list(iter(model.named_parameters()))[0][1].detach().numpy()
b = list(iter(model.named_parameters()))[2][1].detach().numpy()

a.shape, b.shape

((20, 23298), (1, 20))

In [31]:
(a.T @ b.T)[:,0]

array([ 0.00581049,  2.3204684 ,  0.3233345 , ...,  0.0032787 ,
       -0.00736779,  0.02395808], dtype=float32)

In [32]:
parameters = pd.Series((a.T @ b.T)[:, 0])

parameters.sort_values()

69    -2.391297
28    -2.082551
62    -1.812777
40    -0.513019
33    -0.486006
         ...   
631    0.715988
53     0.731409
12     0.774564
109    1.377533
1      2.320468
Length: 23298, dtype: float32

In [33]:
print("Words For Spam: \n")

np.array(vocab.get_itos())[parameters.sort_values()[-200:].index][::-1]

Words For Spam: 



array(['__tag__', 'text/html', '__seperator__', 'httpaddr',
       'charset=iso-number-number', '!', '&nbsp', 'your', 'our', '=',
       'only', 'message', 'best', 'solicitation', 'business',
       'advertisement', '&#number', '.', 'offer', 'online', 'do', 'we',
       'software', 'number%', 'no', 'and/or', '----number--', '?',
       'viagra', 'happy', 'not', 'visit', 'regards', 'man', 'life', 'of',
       '--=_nextpart_numberrfkindysadvnqwnumbernerasdf--', 'order',
       'normal', 'many', 'site', 'e', ',', 'vv', 'removed', 'ebay',
       'want', 'dear', 'way', 'mail', 't', 'happiness', 'like', 'a',
       'website', 'you', 'being', 'professional', 'money', 'store',
       'within', 'easy', 'mr', 'company', 'sincerely', 'goals', 'than',
       'net', 'people', 'never', 'securities', 'box', 'importance',
       'gouranga', 'without', 'suspension', 'free', 'p', 'brings',
       '--=====number=_--', 'just', 'cheap', 'creative', 'cialis',
       'nodnumber', '--qzsoft_directmail_seperat

In [34]:
print("Words For No Spam: \n")

np.array(vocab.get_itos())[parameters.sort_values()[:200].index]

Words For No Spam: 



array(['charset=us-ascii', 'emailaddr', 'text/plain', 'gas', 'at', '-',
       'energy', 'michelle', 'attached', 'i', 'subject', '>',
       'number/number/number', 'new', 'fw', 'please',
       'number-number-number', 'questions', 'tw', 'pm', 'enron', 'sent',
       "'", 'number', 'capacity', 'me', 'cc', 'number-number',
       'message-----', 'on', '-----original', 'friday', 'pipeline',
       'lokay', 'this', 'charset=ansi_xnumber', 'year', 'line', 're',
       'natural', 'thanks', 'power', 'jim', 'contract', 'houston', 'the',
       'two', 'california', 'will', 'might', 's', 'today', 'through',
       'to', 'call', 'monday', 'they', 'said', 'from', 'for', 'am', 'has',
       'so', 'transportation', 'march', 'below', 'daily', '<', 'if',
       'along', 'back', 'share', 'whether', 'changes', 'november',
       'forwarded', 'january', 'fuel', 'later', 'comments', 'end',
       'event', 'report', 'think', 'note', 'august', 'or', 'numberth',
       'doc', 'under', 'interesting', 'again'